In [ ]:
#Dependencies
import pandas as pd
import numpy as np
from random import normalvariate
import plotly.express as px

from cadCAD.configuration.utils import config_sim
from cadCAD.configuration import Experiment
from cadCAD.engine import ExecutionContext, Executor

In [ ]:
#State variables
initial_state = {
    'co2': 400,
    'temperature': 290
}

In [ ]:
#System parameters
system_params = {
    'sun_radiation': [1361],
    'temperature_constant': [1e-4],
    'co2_reflectance_factor': [1e-3],
    'co2_gigatons_to_ppm': [1.2e-1],
    'co2_stdev': [40],
    'heat_dissipation_constant': [2075],
    'co2_annual_emissions': [40, 80, 40, 80, 40, 80, 40, 80],
    'year_of_the_wakening': [0, 0, 10, 10, 50, 50, 100, 100]
}

In [ ]:
assert 1e10 == 1*10**10

In [ ]:
#Policy functions
def p_co2_emissions(params, 
                    subbstep, 
                    state_history, 
                    previous_state):
    # Parameters & variables
    mean = params['co2_annual_emissions']
    std = params['co2_stdev']
    conversion_factor = params['co2_gigatons_to_ppm']
    t_w = params['year_of_the_wakening']
    t = previous_state['timestep']
    
    # Logic
    if t > t_w:
        mean = 0
    else:
        mean = mean
    value = normalvariate(mean, std) * conversion_factor

    # Output
    return {'add_co2': value}

In [ ]:
def p_sun_radiation(params, 
                    substep, 
                    state_history, 
                    previous_state):
    # Parameters & variables
    g = params['temperature_constant']
    a = params['sun_radiation']
    
    # Logic
    temp_change = g * a
    
    # Output
    return {'add_temperature': temp_change}

In [ ]:
def p_earth_cooling(params, 
                    substep, 
                    state_history, 
                    previous_state):
    # Parameters & variables
    g = params['temperature_constant']
    K = params['heat_dissipation_constant']
    T = previous_state['temperature']
    
    # Logic
    temp_change = -(g * K * (T / 300) ** 4)
    
    # Output
    return {'add_temperature': temp_change}

In [ ]:
def p_greenhouse_effect(params, 
                        substep, 
                        state_history, 
                        previous_state):
    # Parameters & variables
    g = params['temperature_constant']
    K = params['heat_dissipation_constant']
    beta = params['co2_reflectance_factor']
    T = previous_state['temperature']
    CO2 = previous_state['co2']
    
    # Logic
    alpha = (1 - np.exp(-beta * CO2))
    temp_change = g * alpha * K * (T / 300) ** 4
    
    # Output
    return {'add_temperature': temp_change}

In [ ]:
#State update functions
def s_co2(params, 
          substep, 
          state_history, 
          previous_state,
          policy_input):
    # Parameters & variables
    current_co2 = previous_state['co2']
    co2_change = policy_input['add_co2']
    
    # Logic
    new_co2 = max(current_co2 + co2_change, 0)
    
    # Output
    return ('co2', new_co2)

In [ ]:
def s_temperature(params, 
                  substep, 
                  state_history, 
                  previous_state,
                  policy_input):
    # Parameters & variables
    current_temp = previous_state['temperature']
    temp_change = policy_input['add_temperature']
    
    # Logic
    new_temp = max(current_temp + temp_change, 0)
    
    # Output
    return ('temperature', new_temp)

In [ ]:
partial_state_update_blocks = [
    {
        'label': 'Temperature dynamics', # Useful metadata to describe our partial state update blocks
        'policies': {
            'sun_radiation': p_sun_radiation,
            'earth_cooling': p_earth_cooling,
            'greenhouse_effect': p_greenhouse_effect
        },
        'variables': {
            'temperature': s_temperature
            
        }
    },
    {
        'label': 'CO2 dynamics', # Useful metadata to describe our partial state update blocks
        'policies': {
            'co2_emissions': p_co2_emissions
        },
        'variables': {
            'co2': s_co2
        }
        
    }
]


In [ ]:
#Configuration
MONTE_CARLO_RUNS = 50
SIMULATION_TIMESTEPS = 100

sim_config = config_sim(
    {
        'N': MONTE_CARLO_RUNS,
        'T': range(SIMULATION_TIMESTEPS),
        'M': system_params,
    }
)

from cadCAD import configs
del configs[:] # Clear any prior configs

experiment = Experiment()
experiment.append_configs(
    sim_configs=sim_config,
    initial_state=initial_state,
    partial_state_update_blocks=partial_state_update_blocks
)

In [ ]:
#Execution
exec_context = ExecutionContext()
run = Executor(exec_context=exec_context, configs=configs)

(system_events, tensor_field, sessions) = run.execute()

In [ ]:
#Simulation output preperation

# Get system events and attribute index
df = (pd.DataFrame(system_events)
        .assign(years=lambda df: df.timestep)
        .assign(temperature_celsius=lambda df: df.temperature - 273)
        .query('timestep > 0')
     )

# Clean substeps
first_ind = (df.substep == 0) & (df.timestep == 0)
last_ind = df.substep == max(df.substep)
inds_to_drop = (first_ind | last_ind)
df = df.loc[inds_to_drop].drop(columns=['substep'])

# Attribute parameters to each row
df = df.assign(**configs[0].sim_config['M'])
for i, (_, n_df) in enumerate(df.groupby(['simulation', 'subset', 'run'])):
    df.loc[n_df.index] = n_df.assign(**configs[i].sim_config['M'])

### Analysis 1: How will the Earth's average temperature develop over the next 100 years, if we keep CO2 emissions unchanged at today’s annual emission levels vs. a doubling of today’s emission levels?

In [ ]:
#System analysis
fig_df = df.query('year_of_the_wakening == 100')

fig = px.scatter(
    fig_df,
    x=fig_df.years,
    y=fig_df.temperature_celsius,
    color=fig_df.co2_annual_emissions.astype(str),
    opacity=0.1,
    trendline="lowess",
    labels={'color': 'Yearly CO2 emissions (Gt)'}
)

fig.show()

In [ ]:
fig_df = df.query('year_of_the_wakening == 100')

fig = px.box(
    fig_df,
    x=fig_df.years,
    y=fig_df.temperature_celsius,
    color=fig_df.co2_annual_emissions.astype(str),
    points=False,
    labels={'color': 'Yearly CO2 emissions (Gt)'}
)

fig.show()

### Analysis 2: How will the rate of annual temperature change develop over the next 100 years if we keep CO2 emissions unchanged at today’s annual emission levels vs. a doubling of today’s emission levels?

In [ ]:
fig_df = (df.query('year_of_the_wakening == 100')
            .assign(annual_temperature_increase=lambda df: df.temperature.diff())
            .query('years > 1'))

fig = px.scatter(
    fig_df,
    x=fig_df.years,
    y=fig_df.annual_temperature_increase,
    opacity=0.1,
    trendline="lowess",
    color=fig_df.co2_annual_emissions.astype(str),
    labels={'color': 'Yearly CO2 emissions (Gt)'}
)

fig.show()